In [1]:
import pandas as pd
from geonamescache import GeonamesCache
import re
#from unidecode import unidecode
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

# 1. Obtener dataset

Empezamos cargando el archivo que contine las diferentes ofertas empleo. Estas ofertas se han obtenido de tres portales de empleo que demandan personas con competencias en Data Science y son los siguientes:
* [reed.co.uk](https://www.reed.co.uk/jobs/data-science-jobs)
* [itjobs.ca](https://www.itjobs.ca/en/search-jobs/?keywords=data+science&search=1)
* [jobscube](https://jobcube.com/Data-Science-jobs)

1. Cargar Data frame

In [2]:
df_jobs=pd.read_json('jobs_ds.json')
df_jobs.head()

,title_html,body_html,li_body,non_li_body
0,Data Science Manager Leeds - Reed.co.uk,Data Science Manager Data Science Manager Up t...,[Leading data science projects under the direc...,Data Science Manager Data Science Manager Up t...
1,Data Science Engineer London - Reed.co.uk,Data Science Engineer Data Science Engineer LO...,[5 Years' experience implementing within Data ...,Data Science Engineer Data Science Engineer LO...
2,Head of Data Science London - Reed.co.uk,Head of Data Science Head of Data ScienceLondo...,[Manage a small team of 3 Data Scientists incl...,Head of Data Science Head of Data ScienceLondo...
3,Python Quantitative Researcher - London- Globa...,Python Quantitative Researcher - London- Globa...,"[Minimum 2+ years' experience, Hands-on progra...",Python Quantitative Researcher - London- Globa...
4,Engineering Lead London - Reed.co.uk,Engineering Lead Engineering Manager | Fin-Tec...,[Lead/manage a diverse team of up to 30+ engin...,Engineering Lead Engineering Manager | Fin-Tec...


$$
\begin{aligned}
\begin{array}{|l|l|}
\hline
\textbf{title_html} & \text{Título de cada oferta de empleo}\\
\hline
\textbf{body_html} & \text{Descripción principal de cada oferta de empleo}\\
\hline
\textbf{li_body} & \text{Contenido de las etiquetas <li> de cada oferta de empleo}\\
\hline
\textbf{non_li_body} & \text{Excluye el contenido de las etiquetas <li> de cada oferta de empleo}\\
\hline
\end{array}
\end{aligned}
$$

2. Filtrar empleos para puestos de Data Science

In [3]:
df_ds=df_jobs[['title_html']][df_jobs.title_html.str.contains(r'Data Scien(ce|tist)',case=False)]

* Imprimit estos títulos

In [4]:
for ele in df_ds.title_html:
        print(ele) 

Data Science Manager Leeds - Reed.co.uk
Data Science Engineer London - Reed.co.uk
Head of Data Science London - Reed.co.uk
Data Scientist London - Reed.co.uk
Senior Data Scientist - London- Spark  AWS  Python  SQL  Scala  Java London - Reed.co.uk
Data Scientist Chester - Reed.co.uk
Data Scientist - Travel Middlesex - Reed.co.uk
Head of Data Science London - Reed.co.uk
Data Scientist  Worthing - Reed.co.uk
Data Scientist Finland - Reed.co.uk
Head of Data Science London - Reed.co.uk
Data Scientist / Engineer - Remote Harpenden - Reed.co.uk
Chief Data Scientist London - Reed.co.uk
Statistical Data Scientist - Remote Harpenden - Reed.co.uk
Head of Data Science London - Reed.co.uk
Data Scientist Growth London - Reed.co.uk
Data Science Lead / A.D of Data Science City of London - Reed.co.uk
Data Scientist Reigate - Reed.co.uk
Data Science Platform Engineer Cardiff - Reed.co.uk
Marketing Data Scientist Reigate - Reed.co.uk
Research Data Scientist London - Reed.co.uk
Team Lead - Data Science Lo

Se puede ver que los títulos contienes los nombres de los tres portales de empleo (Reed.co.uk, ITjobs.ca y Jobcube), vamos a utlizar estos nombres para crear un data frame cuyas columnas contengan la imformación más relevante (ciudad, país y titulo de la oferta de empleo).

Para comenzar, se van a crear dos funciones que serán muy útiles para obtener el data frame.

* Crear función para mostrar títulos de las ofertas de empleo

In [5]:
def print_titles(data):
    for ele in data.title_html[0:5]:
        print(ele) 

* Crear función para filtrar ofertas de empleo

In [6]:
def filter_offer_titles(data,regex='Jobcube'):
    df=data[data.title_html.str.contains(regex,case=False)]
    return df

3. Analizar y limpiar títulos del portal **Jobcube**

In [7]:
usa=filter_offer_titles(df_ds)

In [8]:
print_titles(usa)

Condé Nast - Data Science Manager job in New York NY | Jobcube
Alaska Airlines - Director Data Science job in Seattle WA | Jobcube
Codeup - Data Science Instructor job in San Antonio TX | Jobcube
ezCater - Senior Manager, Sales Data Science and Analytics (Remote) job in Boston MA | Jobcube
T3i, Inc - Data Scientist job in Aurora CO | Jobcube


Estos anuncios empiezan mencionando el nonbre la empresa y como solo necesitamos el título que hace referencia al puesto de trabajo, se va a eliminar.

* Eliminar nombre de empresa

In [9]:
title_usa = usa["title_html"].str.split(" - ",n=1, expand = True)[1]
jobs_usa=pd.DataFrame({'Title':title_usa})
jobs_usa

,Title
1650,Data Science Manager job in New York NY | Jobcube
1651,Director Data Science job in Seattle WA | Jobcube
1652,Data Science Instructor job in San Antonio TX ...
1653,"Senior Manager, Sales Data Science and Analyti..."
1654,Data Scientist job in Aurora CO | Jobcube
...,...
2835,Senior Open-Source Data Scientist Consultant j...
2838,"Product Analyst III, Data Science job in Atlan..."
2844,Data Scientist Intern (Supplyframe) job in Pas...
2846,Sr. Data Scientist - Clearance Required job in...


* Dividir **jobs_usa** en dos columnas que sepearen el título y el nombre del portal

In [10]:
jobs_usa[['title','portal']]=jobs_usa.Title.str.split(pat='|',n=1,expand=True)
jobs_usa

,Title,title,portal
1650,Data Science Manager job in New York NY | Jobcube,Data Science Manager job in New York NY,Jobcube
1651,Director Data Science job in Seattle WA | Jobcube,Director Data Science job in Seattle WA,Jobcube
1652,Data Science Instructor job in San Antonio TX ...,Data Science Instructor job in San Antonio TX,Jobcube
1653,"Senior Manager, Sales Data Science and Analyti...","Senior Manager, Sales Data Science and Analyti...",Jobcube
1654,Data Scientist job in Aurora CO | Jobcube,Data Scientist job in Aurora CO,Jobcube
...,...,...,...
2835,Senior Open-Source Data Scientist Consultant j...,Senior Open-Source Data Scientist Consultant j...,Jobcube
2838,"Product Analyst III, Data Science job in Atlan...","Product Analyst III, Data Science job in Atlan...",Jobcube
2844,Data Scientist Intern (Supplyframe) job in Pas...,Data Scientist Intern (Supplyframe) job in Pas...,Jobcube
2846,Sr. Data Scientist - Clearance Required job in...,Sr. Data Scientist - Clearance Required job in...,Jobcube


* Eliminar la columna **Title**

In [11]:
jobs_usa.drop('Title',axis=1,inplace=True)
jobs_usa

,title,portal
1650,Data Science Manager job in New York NY,Jobcube
1651,Director Data Science job in Seattle WA,Jobcube
1652,Data Science Instructor job in San Antonio TX,Jobcube
1653,"Senior Manager, Sales Data Science and Analyti...",Jobcube
1654,Data Scientist job in Aurora CO,Jobcube
...,...,...
2835,Senior Open-Source Data Scientist Consultant j...,Jobcube
2838,"Product Analyst III, Data Science job in Atlan...",Jobcube
2844,Data Scientist Intern (Supplyframe) job in Pas...,Jobcube
2846,Sr. Data Scientist - Clearance Required job in...,Jobcube


4. Analizar y limpiar títulos del portal **ITjobs**

In [12]:
ca=filter_offer_titles(df_ds,regex='ITjobs')
print_titles(ca)

Data Science Specialist Job in Calgary for Cenovus Energy Inc. | ITjobs.ca
Sr Clinical Data Scientist Job in Anywhere for Syneos Health | ITjobs.ca
Principal/sr Clinical Data Scientist Job in Anywhere for Syneos Health | ITjobs.ca
Lead Data Scientist Job in Toronto for Equifax | ITjobs.ca
Data Scientist Job in Chalk River for Canadian Nuclear Laboratories (CNL) | ITjobs.ca


In [13]:
jobs_ca=ca[['title_html']]
jobs_ca.tail()

,title_html
3865,Data Scientist/analyst - Summer Internship (En...
3975,Data Scientist Job in Edmonton for NAIT (North...
4086,Engineering Data Scientist Job in Calgary for ...
4120,Manager - Forecasting Models - Retail & Small ...
4197,Sr Clinical Data Scientist Job in Anywhere for...


* Dividir **title_html** de **jobs_ca** en dos columnas que sepearen el título y el código del país.

In [14]:
jobs_ca[['title','portal']]=jobs_ca.title_html.str.split(pat='|',n=1,expand=True)
jobs_ca.head()

,title_html,title,portal
2849,Data Science Specialist Job in Calgary for Cen...,Data Science Specialist Job in Calgary for Cen...,ITjobs.ca
2850,Sr Clinical Data Scientist Job in Anywhere for...,Sr Clinical Data Scientist Job in Anywhere for...,ITjobs.ca
2961,Principal/sr Clinical Data Scientist Job in An...,Principal/sr Clinical Data Scientist Job in An...,ITjobs.ca
3072,Lead Data Scientist Job in Toronto for Equifax...,Lead Data Scientist Job in Toronto for Equifax,ITjobs.ca
3183,Data Scientist Job in Chalk River for Canadian...,Data Scientist Job in Chalk River for Canadian...,ITjobs.ca


* Eliminar la columna **title_html**

In [15]:
jobs_ca.drop('title_html',axis=1,inplace=True)
jobs_ca.tail()

,title,portal
3865,Data Scientist/analyst - Summer Internship (En...,ITjobs.ca
3975,Data Scientist Job in Edmonton for NAIT (North...,ITjobs.ca
4086,Engineering Data Scientist Job in Calgary for ...,ITjobs.ca
4120,Manager - Forecasting Models - Retail & Small ...,ITjobs.ca
4197,Sr Clinical Data Scientist Job in Anywhere for...,ITjobs.ca


5. Analizar y limpiar títulos del portal **Reed.co**

In [16]:
uk=filter_offer_titles(df_ds,regex='Reed.co')
print_titles(uk)

Data Science Manager Leeds - Reed.co.uk
Data Science Engineer London - Reed.co.uk
Head of Data Science London - Reed.co.uk
Data Scientist London - Reed.co.uk
Senior Data Scientist - London- Spark  AWS  Python  SQL  Scala  Java London - Reed.co.uk


In [17]:
jobs_uk=uk[['title_html']]
jobs_uk.head()

,title_html
0,Data Science Manager Leeds - Reed.co.uk
1,Data Science Engineer London - Reed.co.uk
2,Head of Data Science London - Reed.co.uk
14,Data Scientist London - Reed.co.uk
36,Senior Data Scientist - London- Spark AWS Py...


* Dividir **title_html** de **jobs_uk** en dos columnas que sepearen el título y el nombre del portal

In [18]:
jobs_uk[['title','portal']]=jobs_uk.title_html.str.split(pat='- R',n=1,expand=True)
jobs_uk

,title_html,title,portal
0,Data Science Manager Leeds - Reed.co.uk,Data Science Manager Leeds,eed.co.uk
1,Data Science Engineer London - Reed.co.uk,Data Science Engineer London,eed.co.uk
2,Head of Data Science London - Reed.co.uk,Head of Data Science London,eed.co.uk
14,Data Scientist London - Reed.co.uk,Data Scientist London,eed.co.uk
36,Senior Data Scientist - London- Spark AWS Py...,Senior Data Scientist - London- Spark AWS Py...,eed.co.uk
...,...,...,...
1517,Junior and Mid level Data Scientists Bristol -...,Junior and Mid level Data Scientists Bristol,eed.co.uk
1539,Data Science Consultant Tadworth - Reed.co.uk,Data Science Consultant Tadworth,eed.co.uk
1562,Senior Financial Data Scientist- Global Hedge ...,Senior Financial Data Scientist- Global Hedge ...,eed.co.uk
1628,Senior / Lead Data Scientist City of London - ...,Senior / Lead Data Scientist City of London,eed.co.uk


* Eliminar la columna **title_html**

In [19]:
jobs_uk.drop('title_html',axis=1,inplace=True)
jobs_uk.tail()

,title,portal
1517,Junior and Mid level Data Scientists Bristol,eed.co.uk
1539,Data Science Consultant Tadworth,eed.co.uk
1562,Senior Financial Data Scientist- Global Hedge ...,eed.co.uk
1628,Senior / Lead Data Scientist City of London,eed.co.uk
1639,"Senior Data Scientist, Renewables Earley",eed.co.uk


6. Concatenar **Jobs_usa**, **Jobs_ca** y **Jobs_uk**

In [20]:
df=pd.concat([jobs_usa,jobs_ca,jobs_uk], axis=0)
df

,title,portal
1650,Data Science Manager job in New York NY,Jobcube
1651,Director Data Science job in Seattle WA,Jobcube
1652,Data Science Instructor job in San Antonio TX,Jobcube
1653,"Senior Manager, Sales Data Science and Analyti...",Jobcube
1654,Data Scientist job in Aurora CO,Jobcube
...,...,...
1517,Junior and Mid level Data Scientists Bristol,eed.co.uk
1539,Data Science Consultant Tadworth,eed.co.uk
1562,Senior Financial Data Scientist- Global Hedge ...,eed.co.uk
1628,Senior / Lead Data Scientist City of London,eed.co.uk


7. Exportar data frame a archivo csv.

## EDA

1. Verificar y eliminar valores nulos

In [26]:
df.isnull().sum()

title     7
portal    7
dtype: int64

In [27]:
df.dropna(inplace=True)
df.head()

,title,portal
1650,Data Science Manager job in New York NY,Jobcube
1651,Director Data Science job in Seattle WA,Jobcube
1652,Data Science Instructor job in San Antonio TX,Jobcube
1653,"Senior Manager, Sales Data Science and Analyti...",Jobcube
1654,Data Scientist job in Aurora CO,Jobcube


In [28]:
df.isnull().sum()

title     0
portal    0
dtype: int64

2. Verificar y eliminar duplicados

In [29]:
df.duplicated(keep=False)

1650     True
1651    False
1652    False
1653    False
1654     True
        ...  
1517    False
1539    False
1562    False
1628    False
1639    False
Length: 956, dtype: bool

In [30]:
df.drop_duplicates(inplace=True)
df

,title,portal
1650,Data Science Manager job in New York NY,Jobcube
1651,Director Data Science job in Seattle WA,Jobcube
1652,Data Science Instructor job in San Antonio TX,Jobcube
1653,"Senior Manager, Sales Data Science and Analyti...",Jobcube
1654,Data Scientist job in Aurora CO,Jobcube
...,...,...
1517,Junior and Mid level Data Scientists Bristol,eed.co.uk
1539,Data Science Consultant Tadworth,eed.co.uk
1562,Senior Financial Data Scientist- Global Hedge ...,eed.co.uk
1628,Senior / Lead Data Scientist City of London,eed.co.uk


3. Exporta el nuevo data frame

In [31]:
df.to_csv('jobs_titles.csv',index=False)